In [235]:
import pandas as pd
import configparser
import sys
import os
import inspect
import evaluation as eval

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [236]:
config = configparser.ConfigParser()
# Read the configuration file
config.read('config.ini')
api_key_openai = config.get('credentials', 'api_key_openai')
api_key_mistral = config.get('credentials', 'api_key_mistral')
surfdrive_url_transcript_sentences = config.get('credentials', 'surfdrive_url_transcript_sentences')

output_evaluation_folder_path = 'output_evaluation/'

### Input

In [237]:
# read output of LLM
fileNoContext = 'Prompts_NoContext_complete_output.csv'
fileContext = 'Prompts_Context_complete_output.csv'

outputNoContext = pd.read_csv(output_evaluation_folder_path+fileNoContext)
outputContext = pd.read_csv(output_evaluation_folder_path+fileContext)
# read rewritten sentences by participants
transcriptSentences = pd.read_csv(surfdrive_url_transcript_sentences).reset_index()[['user', 'original', 'your_text']]

# merge two files
data_for_evaluation_noContext = pd.merge(outputNoContext, transcriptSentences, on=['user', 'original'])
data_for_evaluation_context = pd.merge(outputContext, transcriptSentences, on=['user', 'original'])
print(len(data_for_evaluation_noContext))
print(len(data_for_evaluation_context))

2400
2400
2400


In [238]:
# check and remove empty files

print(data_for_evaluation_noContext['rewritten_sentence'].isna().sum().sum())
data_for_evaluation_noContext = data_for_evaluation_noContext.dropna(subset=['rewritten_sentence'])
print(len(data_for_evaluation_noContext))

print(data_for_evaluation_context['rewritten_sentence'].isna().sum().sum())
data_for_evaluation_context = data_for_evaluation_context.dropna(subset=['rewritten_sentence'])
print(len(data_for_evaluation_context))


41
2359
91
2309


### Evaluation Prompts


In [239]:
prompts_dict = {
    # Generic prompt
    'prompt_llm': 'You are an expert in text style transfer. Here is paragraph S1: {} ',
                 
    # Accuracy
    'prompt_accuracy_s2': 'and paragraph S2: {} ',
    'prompt_accuracy_inference': 'How different is the conversational style of paragraph S2 compared to S1 on a continuous scale from 1 (completely identical styles) to 10 (completely different styles)? Result = . Format result as "score" and "explanation".',
    
    # Content preservation
    'prompt_content_preservation_s2': 'and paragraph S2: {} ',
    'prompt_content_preservation_inference': 'How much does S1 preserve the content of S2 on a continuous scale from 1 (completely different topic) to 10 (identical topic)? Result = .Format result as "score" and "explanation".',
    
    # Fluency
    'prompt_fluency_inference': 'on a scale from 1 to 10 where 1 (lowest coherent) and 10 (highest coherent)? Result = Format result as "score" and "explanation".'
}




In [206]:
row = data_for_evaluation_noContext.iloc[0]
row2 = pd.DataFrame(row)
row2.T

,fileID,user,promptID,model,shots,runID,original,rewritten_sentence,your_text
0,0,U2,0,mistral-medium,5,882824,I’m all about that food. I usually kick off th...,"I'm all about that grub, you know? I usually s...",I’m an absolute foodie! I normally start my we...


### Evaluation using Mistral

In [212]:
# No Context 
# Run00
for index, row in data_for_evaluation_noContext.iterrows():
    print(index)
    name= 'NoContext' + format(index)
    evaluation_noContext_mistral = eval.get_evaluation_mistral(prompts_dict, row, name)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
79
80
81
82
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280


MistralException: Unexpected exception (ReadTimeout): The read operation timed out

In [ ]:
# Context 
# Run00
evaluation_context00_mistral = eval.get_evaluation_mistral(prompts_dict, data_for_evaluation_context, 'Context00')

### Evaluation using GPT

In [ ]:
# No Context 
# Run00
evaluation_noContext00_gpt = eval.get_evaluation_gpt(prompts_dict, data_for_evaluation_noContext, 'NoContext00')

In [ ]:
# Context 
# Run00
evaluation_context00_gpt = eval.get_evaluation_gpt(prompts_dict, data_for_evaluation_context, 'Context00')